In [98]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [99]:
from dataclasses import dataclass

@dataclass
class MetOceanParameters:
    Hm0: float        # Significant wave height [m]
    Tp: float         # Peak wave period [s]
    T02: float        # Mean zero-crossing period [s]
    CS_total: float   # Depth-averaged current speed [m/s]
    WS160: float      # Wind speed at 160 m [m/s]
    WS160_adj: float  # Wind speed at 160 m (+7%) [m/s]
    WL_tot: float     # Total water level [m MSL]
    Hmax: float       # Maximum wave height [m]
    T_Hmax: float     # Wave period associated with Hmax [s]
    Cmax_SWL: float   # Max crest level w.r.t. SWL [m]
    Cmax_MSL: float   # Max crest level w.r.t. MSL [m]
    WL_total: float      # Total water level (HWL_tot or LWL_tot) [mMSL]
    WL_residual: float   # Residual water level (HWL_res or LWL_res) [m]

# -------------------------
# 200-year return period
# -------------------------

HWL = MetOceanParameters(
    Hm0=9.9,
    Tp=15.3,
    T02=9.8,
    CS_total=0.5,
    WS160=29.7,
    WS160_adj=31.8,
    WL_tot=1.3,
    Hmax=19.2,
    T_Hmax=12.1,
    Cmax_SWL=12.8,
    Cmax_MSL=14.6,
    WL_total=2.4,
    WL_residual=2.2
)

LWL = MetOceanParameters(
    Hm0=8.7,
    Tp=13.4,
    T02=8.5,
    CS_total=0.5,
    WS160=32.9,
    WS160_adj=35.2,
    WL_tot=-0.2,
    Hmax=16.6,
    T_Hmax=10.5,
    Cmax_SWL=11.0,
    Cmax_MSL=10.7,
    WL_total=-1.7,
    WL_residual=-1.4
)


for a single layer t_single =  t_static + S_x%  

t_single =  total layer thickness   

t_static = static layer thickness that should always be satisfied for filter functionality  

S_x% = expected scour protection deformation  


minimum required thickness to prevent winnowing 
With:  
 t = layer thickness [m]  

 Dx = Adjusted rock material diameter [m]  

 drel  = Adjusted relative sediment diameter [-]  

 Uc = Depth-averaged current velocity [m/s]  

 Ucrit,KB = Critical velocity in filter, based on Klein Breteler et al. (1992) [m/s]  

 D15 = rock material diameter, at 15% of passing by weight [m]  

 d15 = base material diameter, at 15% of passing by weight [m]  

 D50 = rock material diameter, at 50% of passing by weight [m]  
 

In [100]:
rho_water = 1025  # kg/m3
rho_sediment = 2650  # kg/m3
C1 = 0.4300
X1 = 1.3082
X2 = 4.1017
X3 = -2.5811
d15 = 0.0001
d50 = 0.0003
D15 = 0.187
D50 = 0.20
n = 0.4
c = 0.56
m = 0.15
shields_parameter = 0.044
relative_density = (rho_sediment - rho_water) / rho_water
paal_diameter = 9 # m



assumption for sand and water d15 = m, d50 = m, rho_w = 1025 kg/m^3, rho_sediment = 2650 kg/m^3, v = 1 * 10^-6

Design coefficients in the formula for the minimum required layer thickness from HASPRO: c1 = 0.4300, x1 = 1.3082, x2 = 4.1017, x3 = -2.5811

Coefficients in the formula for the critical filter velocity from HASPRO: Shields_parameter = , c = , m = 

porosity of rock material = 

In [101]:
def d_rel(d15: float, D15: float) -> float:
    """
    d_rel = max(d15 / D15, 1/200)
    """
    return max(d15 / D15, 1.0 / 200.0)


def D_x(D15: float, D50: float) -> float:
    """
    D_x = max(D15, D50 / 1.5)
    """
    return max(D15, D50 / 1.5)


# --------------------------
# Klein Breteler et al. (1992)
# --------------------------

def Ucrit_KB(
    *,
    n: float,      # porositeit [-]
    c: float,      # coefficient c(m) [-]
    m: float,      # exponent m [-]
    D15: float,    # [m]
    nu: float,     # kinematische viscositeit [m2/s]
    psi: float,    # Shields parameter [-]
    delta: float,  # relatieve dichtheid (ρs/ρw − 1) [-]
    d50: float,    # [m]
    g: float = 9.81
) -> float:
    """
    Kritische snelheid in de steenlaag volgens Klein Breteler et al. (1992)

    U_crit,KB =
        [ (n / c) * (D15 / nu)^m * sqrt(psi * g * delta * d50) ]^(1 / (1 - m))
    """

    if nu <= 0 or D15 <= 0 or d50 <= 0:
        raise ValueError("nu, D15 en d50 moeten > 0 zijn.")

    bracket = (
        (n / c)
        * (D15 / nu) ** m
        * math.sqrt(psi * g * delta * d50)
    )

    return bracket ** (1.0 / (1.0 - m))


# --------------------------
# Verhouding t / D_x
# --------------------------

def thickness_ratio(
    *,
    d_rel: float,
    Uc: float,
    Ucrit_KB: float,
    D15: float,
    g: float = 9.81
) -> float:
    """
    t / D_x volgens vergelijking (6)
    """
    return (
        (d_rel ** X1)
        * ((C1 * Uc / Ucrit_KB) ** X2)
        * ((Uc / math.sqrt(g * D15)) ** X3)
    )


# --------------------------
# Eindfunctie (retourneert ALLEEN t)
# --------------------------

def minimum_required_thickness(
    *,
    Uc: float,
    d15: float,
    D15: float,
    D50: float,
    # parameters Klein Breteler
    n: float,
    c: float,
    m: float,
    nu: float,
    psi: float,
    delta: float,
    d50: float,
    g: float = 9.81
) -> float:
    """
    Minimale laagdikte t om winnowing te voorkomen
    """

    # hulpvariabelen
    drel = d_rel(d15, D15)
    Dx = D_x(D15, D50)

    # kritische snelheid volgens KB
    Ucrit = Ucrit_KB(
        n=n,
        c=c,
        m=m,
        D15=D15,
        nu=nu,
        psi=psi,
        delta=delta,
        d50=d50,
        g=g
    )

    # t / Dx
    t_over_Dx = thickness_ratio(
        d_rel=drel,
        Uc=Uc,
        Ucrit_KB=Ucrit,
        D15=D15,
        g=g
    )

    return t_over_Dx * Dx

In [105]:
t = minimum_required_thickness(
    Uc=1.2,
    d15=d15,
    D15=D15,
    D50=D50,
    n=n,
    c=c,
    m=m,
    nu=1.33e-6,
    psi=shields_parameter,
    delta=relative_density,
    d50=d50
    )

print(f"Minimale laagdikte t = {t:.4f} m")

Minimale laagdikte t = 12.4441 m


In [103]:
if 7 * D_x(D15, D50) > t:
    print("Laagdikte voldoet niet aan de eis van 7*D_x.")
print(f"Minimale laagdikte volgens 7*D_x = {7 * D_x(D15, D50):.4f} m")

Minimale laagdikte volgens 7*D_x = 1.3090 m


Radial extend 

In [ ]:
slope = 1/2
h = 
radial_extend = paal_diameter + slope * t + 1 + 1.4 * h
print(f"Radiale uitbreiding van de filterlaag = {radial_extend:.2f} m")

Radiale uitbreiding van de filterlaag = 33.64 m
